In [169]:
import numpy as np
import pandas as pd
import emoji
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tf.compat.v1.data.Iterator
tf.compat.v1.disable_eager_execution()
from keras.layers import Bidirectional, GRU, Dropout

In [170]:
data = pd.read_csv('emoji_data.csv', header = None)
data.head()

,0,1
0,French macaroon is so tasty,4
1,work is horrible,3
2,I am upset,3
3,throw the ball,1
4,Good joke,2


In [171]:
emoji_dict = {
    0: ":red_heart:",
    1: ":baseball:",
    2: ":grinning_face_with_big_eyes:",
    3: ":disappointed_face:",
    4: ":fork_and_knife_with_plate:"
}

def label_to_emoji(label):
    return emoji.emojize(emoji_dict[label])

In [172]:
X = data[0].values
Y = data[1].values
Y = [int(value) if str(value).isdigit() else 0 for value in Y]

# Embeddings

In [173]:
file = open('glove.6B.300d.txt', 'r', encoding = 'utf8')
content = file.readlines()
file.close()

In [174]:
embeddings = {}

for line in content:
    line = line.split()
    embeddings[line[0]] = np.array(line[1:], dtype = float)

In [176]:
def get_maxlen(data):
    maxlen = 0
    for sent in data:
        maxlen = max(maxlen, len(sent))
    return maxlen

maxlen = get_maxlen(X)
print(maxlen)

52


In [177]:
tokenizer = Tokenizer()  
tokenizer.fit_on_texts(X)
word2index = tokenizer.word_index

In [178]:
Xtokens = tokenizer.texts_to_sequences(X)
Xtrain = pad_sequences(Xtokens, maxlen = maxlen,  padding = 'post', truncating = 'post')

In [179]:
Ytrain = to_categorical(Y,num_classes=len(emoji_dict))

# Model

In [180]:
embed_size = 300
embedding_matrix = np.random.rand(len(word2index) + 1, embed_size)

for word, i in word2index.items():
    if word in embeddings:
        embed_vector = embeddings[word]
        embedding_matrix[i] = embed_vector

In [181]:
embedding_matrix

array([[ 0.09329788,  0.65691931,  0.42331902, ...,  0.6594547 ,
         0.14247782,  0.43261318],
       [-0.13292   ,  0.16985   , -0.1436    , ..., -0.23778   ,
         0.14766   ,  0.62902   ],
       [-0.28427   ,  0.047977  , -0.15062   , ..., -0.090071  ,
         0.016922  ,  0.29278   ],
       ...,
       [-0.5211    ,  0.65123   , -0.30188   , ..., -0.23667   ,
        -0.36634   ,  0.16132   ],
       [-0.14154   ,  0.027303  ,  0.13594   , ..., -0.40974   ,
         0.12668   ,  0.11425   ],
       [-0.41355   ,  0.076012  ,  0.099178  , ...,  0.15953   ,
        -0.13223   ,  0.03091   ]])

In [182]:
model = Sequential([
    Embedding(input_dim=len(word2index) + 1,
              output_dim=embed_size,
              input_length=maxlen,
              weights=[embedding_matrix],
              trainable=False
              ),
    
    Bidirectional(GRU(units=64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(GRU(units=32)),
    Dropout(0.5),
    Dense(len(emoji_dict), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [183]:
model.fit(Xtrain, Ytrain, epochs = 100)

Train on 183 samples
Epoch 1/100
183/183 [==============================] - 4s 24ms/sample - loss: 1.6992 - accuracy: 0.2514
Epoch 2/100
183/183 [==============================] - 1s 7ms/sample - loss: 1.5898 - accuracy: 0.2951
Epoch 3/100
183/183 [==============================] - 1s 6ms/sample - loss: 1.5993 - accuracy: 0.2896
Epoch 4/100
183/183 [==============================] - 1s 7ms/sample - loss: 1.4855 - accuracy: 0.3716
Epoch 5/100
183/183 [==============================] - 1s 6ms/sample - loss: 1.4476 - accuracy: 0.3880
Epoch 6/100
183/183 [==============================] - 1s 6ms/sample - loss: 1.3968 - accuracy: 0.4098
Epoch 7/100
183/183 [==============================] - 1s 7ms/sample - loss: 1.4139 - accuracy: 0.4044
Epoch 8/100
183/183 [==============================] - 1s 7ms/sample - loss: 1.2905 - accuracy: 0.4754
Epoch 9/100
183/183 [==============================] - 1s 6ms/sample - loss: 1.2880 - accuracy: 0.4645
Epoch 10/100
183/183 [=============================

In [184]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 52, 300)           93900     
                                                                 
 bidirectional_2 (Bidirecti  (None, 52, 128)           140544    
 onal)                                                           
                                                                 
 dropout_2 (Dropout)         (None, 52, 128)           0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                31104     
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 5)                

In [185]:
test_sentences= ["I feel good", "I feel very bad", "lets eat dinner"]

test_seq = tokenizer.texts_to_sequences(test_sentences)
Xtest = pad_sequences(test_seq, maxlen = maxlen, padding = 'post', truncating = 'post')

y_pred = model.predict(Xtest)
y_pred = np.argmax(y_pred, axis = 1)

for i in range(len(test_sentences)):
    print(test_sentences[i], label_to_emoji(y_pred[i]))

c:\Users\RIYA\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


I feel good 😃
I feel very bad 😞
lets eat dinner 🍽️
